Install yolo

In [3]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [4]:
with open('CVC_challenge_env\\Lib\\site-packages\\ultralytics\\cfg\\datasets\\coco128.yaml', 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)


# Ultralytics YOLO 🚀, AGPL-3.0 license
# COCO128 dataset https://www.kaggle.com/datasets/ultralytics/coco128 (first 128 images from COCO train2017) by Ultralytics
# Documentation: https://docs.ultralytics.com/datasets/detect/coco/
# Example usage: yolo train data=coco128.yaml
# parent
# ├── ultralytics
# └── datasets
#     └── coco128  ← downloads here (7 MB)

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../datasets/coco128 # dataset root dir
train: images/train2017 # train images (relative to 'path') 128 images
val: images/train2017 # val images (relative to 'path') 128 images
test: # test images (optional)

# Classes
names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  

recognision on image

In [5]:
!yolo predict model=yolo11s.pt source='image.jpg' conf=0.5 iou=0.5 #threshold = 0.5 and non-max suppression = 0.5

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\CVC_challenge_env\Scripts\yolo.exe\__main__.py", line 7, in <module>
  File "c:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\CVC_challenge_env\Lib\site-packages\ultralytics\cfg\__init__.py", line 754, in entrypoint
    check_dict_alignment(full_args_dict, {a: ""})
  File "c:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\CVC_challenge_env\Lib\site-packages\ultralytics\cfg\__init__.py", line 436, in check_dict_alignment
    raise SyntaxError(string + CLI_HELP_MSG) from e
SyntaxError: 'and' is not a valid YOLO argument. 

    Arguments received: ['yolo', 'predict', 'model=yolo11s.pt', "source='image.jpg'", 'conf=0.5', 'iou=0.5', '#threshold', '=', '0.5', 'and', 'non-max', 'suppression', '=', '0.5']. Ultralytics 'yolo' commands use the follow

In [6]:
# segmentation
# !yolo task=segment mode=predict model=yolo11s-seg.pt source='image.jpg'

Ultralytics 8.3.18 🚀 Python-3.12.0 torch-2.5.0+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
YOLO11s-seg summary (fused): 265 layers, 10,097,776 parameters, 0 gradients, 35.5 GFLOPs

image 1/1 c:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\image.jpg: 416x640 3 cars, 269.8ms
Speed: 5.2ms preprocess, 269.8ms inference, 15.2ms postprocess per image at shape (1, 3, 416, 640)
Results saved to c:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\runs\segment\predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


Custom yolo

In [4]:
#train
!yolo task=detect mode=train model=yolo11s.pt data=CVC_v1i_yolov11/data.yaml epochs=5


Ultralytics 8.3.18 🚀 Python-3.12.0 torch-2.5.0+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=CVC_v1i_yolov11/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf


  0%|          | 0.00/755k [00:00<?, ?B/s]
100%|██████████| 755k/755k [00:00<00:00, 7.20MB/s]
100%|██████████| 755k/755k [00:00<00:00, 7.20MB/s]

train: Scanning C:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\CVC_v1i_yolov11\train\labels...:   0%|          | 0/30 [00:00<?, ?it/s]
train: Scanning C:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\CVC_v1i_yolov11\train\labels... 22 images, 0 backgrounds, 0 corrupt:  73%|███████▎  | 22/30 [00:00<00:00, 202.74it/s]
train: Scanning C:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\CVC_v1i_yolov11\train\labels... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<00:00, 254.75it/s]

val: Scanning C:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\CVC_v1i_yolov11\valid\labels...:   0%|          | 0/3 [00:00<?, ?it/s]
val: Scanning C:\Users\Gebruiker\OneDrive - Thomas More\Bureaublad\CVC_challenge\CVC_v1i_yolov11\valid\labels... 3 images, 0 backgrounds, 0 cor

OD in game

In [ ]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import cv2 as cv
import os
import random

In [ ]:
class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img) 
            
        return img

    def generate_image_dataset(self):
        if not os.path.exists("images"):
            os.mkdir("images")
        while(True):
            img = self.get_screenshot()
            im = Image.fromarray(img[..., [2, 1, 0]])
            im.save(f"./images/img_{len(os.listdir('images'))}.jpeg")
            sleep(1)
    
    def get_window_size(self):
        return (self.w, self.h)
    


In [ ]:
import torch

class ImageProcessor:
    def __init__(self, window_size, model_path):
        # Load YOLOv11 model from a given path
        self.model = torch.hub.load('ultralytics/yolov11', 'custom', path=model_path)  # YOLOv11 from Ultralytics
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device).eval()  # Put the model in evaluation mode
        
    def process_image(self, img):
        # Preprocess the image for YOLOv11
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img_tensor = torch.from_numpy(img).float().permute(2, 0, 1) / 255.0
        img_tensor = img_tensor.unsqueeze(0).to(self.device)
        
        # Perform inference
        with torch.no_grad():
            detections = self.model(img_tensor)
        
        # Extract bounding boxes, confidence, and class predictions
        results = detections.xyxy[0].cpu().numpy()
        coordinates = []
        for x1, y1, x2, y2, conf, cls in results:
            coordinates.append({
                'x': int(x1),
                'y': int(y1),
                'w': int(x2 - x1),
                'h': int(y2 - y1),
                'class': int(cls),
                'confidence': float(conf),
                'class_name': self.model.names[int(cls)]
            })
        return coordinates


In [ ]:
window_name = "Forza Horizon 4"
model_path = "yolo11s.pt"  # Replace with your YOLOv11 model path

wincap = WindowCapture(window_name)
improc = ImageProcessor(wincap.get_window_size(), model_path)

while True:
    ss = wincap.get_screenshot()

    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break

    coordinates = improc.process_image(ss)
    
    for coordinate in coordinates:
        print(coordinate)
    print()
    
    # If you have limited computer resources, consider adding a sleep delay between detections.
    # sleep(0.2)

In [ ]:
    # Chat gpt code
    
# import win32gui, win32ui, win32con, win32api
# from PIL import Image

# def capture_screen(region=None):
#     hwin = win32gui.GetDesktopWindow()
#     if region:
#         left, top, right, bot = region
#         width = right - left
#         height = bot - top
#     else:
#         width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
#         height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
#         left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
#         top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

#     hwindc = win32gui.GetWindowDC(hwin)
#     srcdc = win32ui.CreateDCFromHandle(hwindc)
#     memdc = srcdc.CreateCompatibleDC()
#     bmp = win32ui.CreateBitmap()
#     bmp.CreateCompatibleBitmap(srcdc, width, height)
#     memdc.SelectObject(bmp)
#     memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)

#     bmpinfo = bmp.GetInfo()
#     bmpstr = bmp.GetBitmapBits(True)
#     img = Image.frombuffer('RGB', (bmpinfo['bmWidth'], bmpinfo['bmHeight']), bmpstr, 'raw', 'BGRX', 0, 1)

#     win32gui.DeleteObject(bmp.GetHandle())
#     memdc.DeleteDC()
#     srcdc.DeleteDC()
#     win32gui.ReleaseDC(hwin, hwindc)

#     return img

In [ ]:
# from ultralytics import YOLO
# import cv2
# import numpy as np

# # Load your YOLO model
# model = YOLO('yolo11s.pt')  # or any other YOLO model you use

# def detect_objects(frame):
#     # Convert PIL image to OpenCV format
#     frame = np.array(frame)
#     frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

#     # Perform YOLO prediction
#     results = model(frame)

#     # Visualize or process results
#     annotated_frame = results[0].plot()  # YOLO will return an annotated frame
#     return annotated_frame

# # Capture a screen region (game window)
# region = (0, 0, 800, 600)  # Adjust to your game window size
# while True:
#     screen = capture_screen(region)
#     detected_frame = detect_objects(screen)

#     # Display the detection
#     cv2.imshow("Game Object Detection", detected_frame)

#     # Exit on 'q' key
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cv2.destroyAllWindows()


In [ ]:
# def detect_objects(frame):
#     # Resize frame before detection for faster inference
#     frame_resized = cv2.resize(frame, (640, 640))

#     # Perform YOLO prediction
#     results = model(frame_resized)

#     # Return results (annotated frame)
#     return results[0].plot()
